In [1]:
def fxses(date, security):
    x = dt.datetime.timestamp(dt.datetime.strptime(str(date), '%Y-%m-%d %H:%M:%S'))
    element = dt.datetime.fromtimestamp(x).strftime('%Y-%m-%d')
    y = dt.datetime.strptime(element, '%Y-%m-%d')
    y_ts = dt.datetime.timestamp(y)

    # above codes, capture and change current time to timestamp
    # Convert the timestamp into YYYY-MM-DD format, in a way stripping the HH:MM:SS elements
    # capture the timestamp of the stripped date data, it would arrive YYYY-MM-DD, 00:00:00
    
    if y.weekday() == 6: # if the date was a Sunday, refer 2 days back, which is Friday
        tokyo_0 = y_ts - 172800 - 3600
        tokyo_1 = y_ts - 172800 + 28800
        london_0 = y_ts - 172800 + 25200
        london_1 = y_ts - 172800 + 57600
        newyork_0 = y_ts - 172800 + 43200
        newyork_1 = y_ts - 172800 + 72000
    elif y.weekday() == 0: # if the date was a Monday, refer 3 days back, which is Friday
        tokyo_0 = y_ts - 259200 - 3600
        tokyo_1 = y_ts - 259200 + 28800
        london_0 = y_ts - 259200 + 25200
        london_1 = y_ts - 259200 + 57600
        newyork_0 = y_ts - 259200 + 43200
        newyork_1 = y_ts - 259200 + 72000
    else: 
        tokyo_0 = y_ts - 86400 - 3600
        tokyo_1 = y_ts - 86400 + 28800
        london_0 = y_ts - 86400 + 25200
        london_1 = y_ts - 86400 + 57600
        newyork_0 = y_ts - 86400 + 43200
        newyork_1 = y_ts - 86400 + 72000
        
    # above using timestamp to segment out the timestamp for the respective hours of the previous day
    # value of 1 represent 1 second, so convert to hours accordingly
    
    tokyo_b = dt.datetime.fromtimestamp(tokyo_0)
    tokyo_e = dt.datetime.fromtimestamp(tokyo_1)
    london_b = dt.datetime.fromtimestamp(london_0)
    london_e = dt.datetime.fromtimestamp(london_1)
    newyork_b = dt.datetime.fromtimestamp(newyork_0)
    newyork_e = dt.datetime.fromtimestamp(newyork_1)
    
    # above convert the timestamp to datetime value
    
    tokyo_high = markets.loc[tokyo_b:tokyo_e]['MID_HIGH'].max()
    tokyo_low = markets.loc[tokyo_b:tokyo_e]['MID_LOW'].min()
    london_high = markets.loc[london_b:london_e]['MID_HIGH'].max()
    london_low = markets.loc[london_b:london_e]['MID_LOW'].min()
    newyork_high = markets.loc[newyork_b:newyork_e]['MID_HIGH'].max()
    newyork_low = markets.loc[newyork_b:newyork_e]['MID_LOW'].min()
    
    # above using reuters API to capture the high & low in hourly interval
    
    if newyork_high > london_high > tokyo_high and newyork_low > london_low > tokyo_low:
        return 'Long'
    elif newyork_high < london_high < tokyo_high and newyork_low < london_low < tokyo_low:
        return 'Short'
    else: return False
    
    # above is the trading rule, the key in this whole exercise
    # target to return trade signal, if neither criteria fulfilled, return False
    # False indicating no trade
    

In [2]:
def sameday(i):
    
    # converting the timestamp to datetime object and extract the date values
    x = dt.datetime.strptime(str(markets.index[i-1]), '%Y-%m-%d %H:%M:%S').date()
    y = dt.datetime.strptime(str(markets.index[i]), '%Y-%m-%d %H:%M:%S').date()
    
    # return true if it is the same date
    return (x == y)

In [3]:
import refinitiv.data as rd
import datetime as dt
import pandas as pd

security = 'JPY='
date_0 = '2022-12-01'
date_1 = '2023-03-31'

rd.open_session()
markets = rd.get_history(universe=[security], fields=['MID_HIGH','MID_LOW'],
                     interval="1h", start=date_0, end=date_1
                    )
# input parameter to generate the market data

In [4]:
# below to test out the logic
# making sure the code run through all the data point
# and FXSES is generating the correct signal for trade

for i in range(24, len(markets)):
    # if same day, next iteration (no results)
    if sameday(i) == False:
        if dt.datetime.strptime(str(markets.index[i]), '%Y-%m-%d %H:%M:%S').date().weekday() == 6: # a brute force way to resolve issue on weekend
            continue # if same day and if Sat, next iteration (no results)
                     # proceed to run FXSES if not same day and not weekends
        elif fxses(markets.index[i], security) == False:
            continue # next iteration if FXSES shows False (no results)
        else: print(markets.index[i], 'Trade') # This is where the algo detect trading opp.
    else: continue
    

2022-12-06 00:00:00 Trade
2022-12-08 00:00:00 Trade
2022-12-13 00:00:00 Trade
2022-12-16 00:00:00 Trade
2022-12-19 00:00:00 Trade
2022-12-21 00:00:00 Trade
2022-12-30 00:00:00 Trade
2023-01-02 00:00:00 Trade
2023-01-13 00:00:00 Trade
2023-01-31 00:00:00 Trade
2023-02-07 00:00:00 Trade
2023-02-08 00:00:00 Trade
2023-02-16 00:00:00 Trade
2023-02-27 00:00:00 Trade
2023-03-06 00:00:00 Trade
2023-03-20 00:00:00 Trade
2023-03-22 00:00:00 Trade
2023-03-28 00:00:00 Trade
2023-03-30 00:00:00 Trade


In [5]:
# based on FXSES signal, generate trade entry and check its accuracy

entry_dict = {}

for i in range(24, len(markets)):
    if sameday(i) == False:
        if dt.datetime.strptime(str(markets.index[i]), '%Y-%m-%d %H:%M:%S').date().weekday() == 6: # a brute force way to resolve issue on weekend
            continue
        elif fxses(markets.index[i], security) == False:
            continue
        elif fxses(markets.index[i], security) == 'Long':
            entry_dict[markets.index[i]] = markets['MID_HIGH'][i]
        elif fxses(markets.index[i], security) == 'Short':
            entry_dict[markets.index[i]] = -markets['MID_LOW'][i]
    else: continue

In [6]:
display(entry_dict)

{Timestamp('2022-12-06 00:00:00'): 136.635,
 Timestamp('2022-12-08 00:00:00'): -136.265,
 Timestamp('2022-12-13 00:00:00'): 137.9,
 Timestamp('2022-12-16 00:00:00'): 137.79,
 Timestamp('2022-12-19 00:00:00'): -135.895,
 Timestamp('2022-12-21 00:00:00'): -131.495,
 Timestamp('2022-12-30 00:00:00'): -132.425,
 Timestamp('2023-01-02 00:00:00'): -130.925,
 Timestamp('2023-01-13 00:00:00'): -128.685,
 Timestamp('2023-01-31 00:00:00'): 130.505,
 Timestamp('2023-02-07 00:00:00'): 132.63,
 Timestamp('2023-02-08 00:00:00'): -130.725,
 Timestamp('2023-02-16 00:00:00'): 134.125,
 Timestamp('2023-02-27 00:00:00'): 136.56,
 Timestamp('2023-03-06 00:00:00'): -135.805,
 Timestamp('2023-03-20 00:00:00'): -132.195,
 Timestamp('2023-03-22 00:00:00'): 132.705,
 Timestamp('2023-03-28 00:00:00'): 131.245,
 Timestamp('2023-03-30 00:00:00'): 132.73}

In [13]:
# started off with no trade
entry = 0
SL = 0 
TP = 0
pnl = []
trade = {}

for i in range(24, len(markets)): # if no trade
    if entry == 0:
        if sameday(i) == False:
            if dt.datetime.strptime(str(markets.index[i]), '%Y-%m-%d %H:%M:%S').date().weekday() == 6: # a brute force way to resolve issue on weekend
                continue
            elif fxses(markets.index[i], security) == False:
                continue
            elif fxses(markets.index[i], security) == 'Long':
                entry = markets['MID_HIGH'][i]
                SL = markets['MID_LOW'][i]
                TP = entry + (entry - SL)
                trade[markets.index[i]] = entry
            elif fxses(markets.index[i], security) == 'Short':
                entry = -markets['MID_LOW'][i]
                SL = markets['MID_HIGH'][i]
                TP = -entry + (-entry - SL)
                trade[markets.index[i]] = entry
        else: continue
    elif entry > 0: # if long
        if markets['MID_HIGH'][i] >= TP:
            pnl.append(markets['MID_HIGH'][i] - entry)
            entry = 0
            SL = 0
            TP = 0
            continue
        elif markets['MID_LOW'][i] <= SL:
            pnl.append(markets['MID_LOW'][i] - entry)
            entry = 0
            SL = 0
            TP = 0
            continue
        else: continue
    elif entry < 0: # if short
        if markets['MID_HIGH'][i] >= SL:
            pnl.append(-entry - markets['MID_HIGH'][i])
            entry = 0
            SL = 0
            TP = 0
            continue
        elif markets['MID_LOW'][i] <= TP:
            pnl.append(-entry - markets['MID_LOW'][i])
            entry = 0
            SL = 0
            TP = 0
            continue
        else: continue
    else: continue
        

In [14]:
display(trade)

{Timestamp('2022-12-06 00:00:00'): 136.635,
 Timestamp('2022-12-08 00:00:00'): -136.265,
 Timestamp('2022-12-13 00:00:00'): 137.9,
 Timestamp('2022-12-16 00:00:00'): 137.79,
 Timestamp('2022-12-19 00:00:00'): -135.895,
 Timestamp('2022-12-21 00:00:00'): -131.495,
 Timestamp('2022-12-30 00:00:00'): -132.425,
 Timestamp('2023-01-02 00:00:00'): -130.925,
 Timestamp('2023-01-13 00:00:00'): -128.685,
 Timestamp('2023-01-31 00:00:00'): 130.505,
 Timestamp('2023-02-07 00:00:00'): 132.63,
 Timestamp('2023-02-08 00:00:00'): -130.725,
 Timestamp('2023-02-16 00:00:00'): 134.125,
 Timestamp('2023-02-27 00:00:00'): 136.56,
 Timestamp('2023-03-06 00:00:00'): -135.805,
 Timestamp('2023-03-20 00:00:00'): -132.195,
 Timestamp('2023-03-22 00:00:00'): 132.705,
 Timestamp('2023-03-28 00:00:00'): 131.245,
 Timestamp('2023-03-30 00:00:00'): 132.73}

In [17]:
display(pnl)

[0.5400000000000205,
 -0.7800000000000011,
 -0.575000000000017,
 -0.49500000000000455,
 -0.8499999999999943,
 -0.8199999999999932,
 0.710000000000008,
 0.160000000000025,
 -0.6699999999999875,
 -0.4000000000000057,
 -0.3949999999999818,
 -0.5150000000000148,
 -0.4900000000000091,
 -0.4650000000000034,
 0.4350000000000023,
 -0.46000000000000796,
 -0.45000000000001705,
 -0.6800000000000068,
 -0.375]

19
